In [ ]:
from yftools import Data
import numpy as np
import pandas as pd
import logging
import plotly.io as pio
import plotly.graph_objs as go
from plotly.subplots import make_subplots
pio.renderers.default = 'notebook'

class Priceinsights:
    def __init__(self,stock=None,startdate=None):
        self.startdate = startdate
        self.stock = stock
        self.logger = logging.getLogger(__name__)
        self.benchm,self.df = self._get_data('^SPX'),self._get_data(stock)
        
    def _get_data(self,stock):
        df = pd.DataFrame()
        if stock is not None:
            df = Data(stock).ohlcv()
            if self.startdate is not None:
                try:
                    df = df.loc[df.index >= pd.to_datetime(self.startdate)]
                except TypeError:
                    self.logger.warning('Invalid Startdate.')
        return df
    
    def plot(self,period=13):
        if self.df.empty:
            return
        
        sr_df = pd.DataFrame(columns=['Benchmark',self.stock])
        
        tickers = {'Benchmark':self.benchm,self.stock:self.df}
        for key,df in tickers.items():
            ret = df['Close'].pct_change().dropna()
            std = ret.rolling(period).std().dropna()*np.sqrt(52)
            mean = ret.rolling(period).mean().dropna()*52
            sr = (mean-0.04)/std
            sr_df[key] = sr
        
        sr_df['Diff'] = sr_df[self.stock]-sr_df['Benchmark']
        
        fig = go.Figure()
        fig.add_trace(
            go.Scatter(
                x=sr_df.index,
                y=sr_df['Diff'],
                name='Difference',
                mode='lines',
                line=dict(color='rgb(241, 196, 15)'),
                fill='tozeroy',
                fillcolor='rgba(241, 196, 15, 0.3)',
                hovertemplate='<b>%{x|%Y-%m-%d}</b><br>Difference: %{y:.3f}<extra></extra>'
            )
        )
        fig.update_layout(
            template='plotly_dark',
            font=dict(size=12,family='Arial, sans-serif'),
            title=dict(text='📊 Sharpe Ratio Difference',
                       font=dict(size=18,weight='bold'),
                       x=0.5),
            yaxis_title='Difference',
            hovermode='x unified',
            height=600
        )
        fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='rgba(255,255,255,0.1)')
        fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(255,255,255,0.1)')
        fig.add_hline(y=0, line_dash="dash", line_color="white", opacity=0.3,row=1,col=1)
        fig.show()
        
if __name__ == '__main__':
    stock = 'BSX'
    sdate = '2023-01-01'
    Priceinsights(stock,sdate).plot()